In [1]:
pip install colorama cryptography

In [2]:
import re
import colorama
import csv
from cryptography.fernet import Fernet
from itertools import product
import base64
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import time

# Q1 - 1

In [4]:
special_chars = ["!", "\"", "#", "$", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<", "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~"]
with open('/content/1000000-password-seclists.txt', 'rb') as file:
    attackers_password = []
    for line in file:
        attackers_password.append((line.strip()).decode('utf-8'))

strength = 5
password = input("Type in your password: ")

dic_flag = False
for p in attackers_password:
  if p in password.lower():
    print(colorama.Fore.RED + '•Password found in the attackers dictionary')
    strength -= 1
    dic_flag = True
    break

if len(password) > 8:
  low_flag, cap_flag, spec_flag, dig_flag = False, False, False, False
  for char in password:
    if char.islower():
      low_flag = True
    elif char.isupper():
      cap_flag = True
    elif char in special_chars:
      spec_flag = True
    elif char.isdigit():
      dig_flag = True
    else:
      raise Exception("Invalid Character")
  if low_flag is False:
    print(colorama.Fore.RED + '•Password must contain at least one lowercase letter')
    strength -= 1
  if cap_flag is False:
    print(colorama.Fore.RED + '•Password must contain at least one uppercase letter')
    strength -= 1
  if dig_flag is False:
    print(colorama.Fore.RED + '•Password must contain at least one digit')
    strength -= 1
  if spec_flag is False:
    print(colorama.Fore.RED + '•Password must contain at least one special letter')
    strength -= 1

else:
  print(colorama.Fore.RED + 'Use 8 characters or more for your password')

if strength == 4 or strength == 5:
  print(colorama.Fore.GREEN)
else:
  print(colorama.Style.RESET_ALL)

if len(password) > 8:
  print("Password Strength: [" + str(strength) + "/5]")



Type in your password: 123asdqwerty
•Password found in the attackers dictionary
•Password must contain at least one uppercase letter
•Password must contain at least one special letter

Password Strength: [2/5]


# Q1 - 2

In [5]:
def brute_force(length, password, chars, known_first='', known_chars=''):
  guessed_password = None
  guess = 0
  start = time.time()
  i = 1
  if known_first != '':
    i = 0
  for l in range(int(length) + i):
      to_attempt = product(chars, repeat=l)
      for attempt in to_attempt:
          curr_guess = known_first + ''.join(attempt)
          if known_chars != '':
            for ch in known_chars:
              if ch not in curr_guess:
                continue
          guess += 1
          if (curr_guess) == password:
            guessed_password = attempt
            break
  if guessed_password:
    print("Guessed password: " + known_first + ''.join(guessed_password))
  else:
    print("Could not crack the password")
  print("Total guesses: " + str(guess))
  end = time.time()
  print("Execution Time: " + str(end - start))


In [9]:
length = input("Enter Password Length: ")
password = input("Enter Password : ")
known_first = input("Enter the first character of your password or press enter to leave it empty : ")
known_chars = input("Enter the known characters of your password or press enter to leave it empty : ")

chars = ' !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~'

if known_chars != '':
  for char in known_chars:
    chars.replace(char, '')
    chars = char + chars

brute_force(length, password, chars, known_first, known_chars)

Enter Password Length: 4
Enter Password : as@1
Enter the first character of your password or press enter to leave it empty : 
Enter the known characters of your password or press enter to leave it empty : s@
Guessed password: as@1
Total guesses: 62080700
Execution Time: 29.91442108154297


# Q2

In [10]:
def csv_printer(file_path):
  with open(file_path) as file:
      content = csv.reader(file)
      for row in content:
          print(row)

In [11]:
def generate_key_from_input(inp):
    salt = b'MyFixedSalt'

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=480000,
    )
    inp = inp.encode('utf-8')
    key = base64.urlsafe_b64encode(kdf.derive(inp))
    return key

In [12]:
print("Before Encryption: ")
csv_printer('/content/country_full.csv')

Before Encryption: 
['name', 'alpha-2', 'alpha-3', 'country-code', 'iso_3166-2', 'region', 'sub-region', 'intermediate-region', 'region-code', 'sub-region-code', 'intermediate-region-code']
['Afghanistan', 'AF', 'AFG', '004', 'ISO 3166-2:AF', 'Asia', 'Southern Asia', '', '142', '034', '']
['Åland Islands', 'AX', 'ALA', '248', 'ISO 3166-2:AX', 'Europe', 'Northern Europe', '', '150', '154', '']
['Albania', 'AL', 'ALB', '008', 'ISO 3166-2:AL', 'Europe', 'Southern Europe', '', '150', '039', '']
['Algeria', 'DZ', 'DZA', '012', 'ISO 3166-2:DZ', 'Africa', 'Northern Africa', '', '002', '015', '']
['American Samoa', 'AS', 'ASM', '016', 'ISO 3166-2:AS', 'Oceania', 'Polynesia', '', '009', '061', '']
['Andorra', 'AD', 'AND', '020', 'ISO 3166-2:AD', 'Europe', 'Southern Europe', '', '150', '039', '']
['Angola', 'AO', 'AGO', '024', 'ISO 3166-2:AO', 'Africa', 'Sub-Saharan Africa', 'Middle Africa', '002', '202', '017']
['Anguilla', 'AI', 'AIA', '660', 'ISO 3166-2:AI', 'Americas', 'Latin America and the

In [20]:
key = input("Enter your Key: ")
safe_key = generate_key_from_input(key)
fernet = Fernet(safe_key)

with open('/country_full.csv', 'rb') as file:
    original = file.read()

encrypted = fernet.encrypt(original)

with open('/country_full_encrypted.csv', 'wb') as encrypted_file:
    encrypted_file.write(encrypted)

print("After Encryption: ")
csv_printer('/country_full_encrypted.csv')

Enter your Key: ASDasd123
After Encryption: 
['gAAAAABlT8748AocOrdYN3RPugJ2s6S61W9LYLOz8R_a_DhQ7eb_4V1MCOD5InzDlZywp-bsxTFG6-NuCNs1rG8M57qQmNjyvF8cUI2Rk3jxsht1QfESQJWo5CCZO9iljse0wPzbKmSZ22yz0AdmInEfvYlh_YnlcL9GO7medBEos8CUJX-kOKRQQm3BeW-eaLf2Ky2XTsfT2ja2iXf1uIlGgcVLZ09fjNCcf9mc_js3TAbws57e7H_llUtUqrF4TkEdmb8zjRWLaJiwoWBSc4_UiBcJvbLgTRgTWfDOD1R2mCataNbOeHP9_wYjePp4vTrILzIW8Kq1n4XCRdYqTZCPxGYr_7TYolKjXhzmoyun5dc8-nNKRn-c3Rq071xKp0PLdKHb-jEzw1oxr2QDoqO-D6GsboF64LuMwOxWzlK1e9eOWEPKOEH7f6oTdkoePeLqEatXXQ7PKvLVz7KZKjijtT2CqonbOFUI4ChJNUm8ndTpvTvWzwNZCQifS_1m1Z3MPpggCYcRN9sE1IPEZ5yQPVXY6ejLEhtAMdV7AGW0u6Hm7-I--9vwsWml9PDzG432-0Jsno483Rrv3GvXPW_dFoug6hrqHNz-9vF4qFcRj5lvXLkhgWB3CdWvOVOGmJgnHMsyVX8QGXUDcl1uC7oJKcA-l7hmCGAZVsnZJ1FeMksQODDRvuuIBLae_5OgS9U1RS143ws04A4gfjb5uH1dRj6dBPjDAMq6_32gpfuUn-TsxOIZAVGbfMuUUC_yRLaS9_q-lzP9P2HzAmU4yhgkptwp8BeiMQ4wSAKMYSL1Bqrxy2B-nBpQJXxPr7a1MC8Sozci8RhOItyIEA5di8q9LSJJTYYn-KOsQs6bLFQucsOih6ZnDWLgGOzaAAJjH9-izbqfBDedj0o2CZWfj9iJGC0qBdBGOfTBJDotfoa7fcDDpNfXNspdZ

In [22]:
key = input("Enter your Key: ")
safe_key = generate_key_from_input(key)
fernet = Fernet(safe_key)

with open('/country_full_encrypted.csv', 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = fernet.decrypt(encrypted)

with open('/country_full_decrypted.csv', 'wb') as decrypted_file:
    decrypted_file.write(decrypted)

print("After decryption:: ")
print(decrypted_file)
csv_printer('/country_full_decrypted.csv')

Enter your Key: ASDasd123
After decryption:: 
<_io.BufferedWriter name='/country_full_decrypted.csv'>
['name', 'alpha-2', 'alpha-3', 'country-code', 'iso_3166-2', 'region', 'sub-region', 'intermediate-region', 'region-code', 'sub-region-code', 'intermediate-region-code']
['Afghanistan', 'AF', 'AFG', '004', 'ISO 3166-2:AF', 'Asia', 'Southern Asia', '', '142', '034', '']
['Åland Islands', 'AX', 'ALA', '248', 'ISO 3166-2:AX', 'Europe', 'Northern Europe', '', '150', '154', '']
['Albania', 'AL', 'ALB', '008', 'ISO 3166-2:AL', 'Europe', 'Southern Europe', '', '150', '039', '']
['Algeria', 'DZ', 'DZA', '012', 'ISO 3166-2:DZ', 'Africa', 'Northern Africa', '', '002', '015', '']
['American Samoa', 'AS', 'ASM', '016', 'ISO 3166-2:AS', 'Oceania', 'Polynesia', '', '009', '061', '']
['Andorra', 'AD', 'AND', '020', 'ISO 3166-2:AD', 'Europe', 'Southern Europe', '', '150', '039', '']
['Angola', 'AO', 'AGO', '024', 'ISO 3166-2:AO', 'Africa', 'Sub-Saharan Africa', 'Middle Africa', '002', '202', '017']
['